In [ ]:
#THE FOLLOWING CODE IS A COMBINATION OF DIFFERENT FILES 
#FROM AUTHORS ABDUL, MURTUZA, DEVESH
#THE CODE HAS BEEN MOTIVATED FROM DIFFERENT LINKS PROVIDED IN THE REPORT

#FILTERING OF THE DATASET
#Importing the libraries
import os
import json
import shutil # for copying the files
import os.path # for setting the destination folder
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os, shutil, glob, os.path
from sklearn.cluster import AgglomerativeClustering

# importing the library for extracting the winrar files
try:
    import lzma
except ImportError:
    from backports import lzma



###############################################################################

# getting current directory
print(os.getcwd())

# main folder having all hash tags folders
main_folder = "D:\\Instagram Data"

# changing cueent directory to main directory
os.chdir(main_folder)

# writing the folder names in the main folder
folders = ['#health', '#medicine']

# iterating each folder in the main folder
for folder in folders:
    # making full apth for the sub folder
    full_path = main_folder+"\\"+folder
    # changing the working directory to the subfolder to work on a particulat hash tag folder
    os.chdir(full_path)
    print(os.getcwd())
    folder_contents_names = [name for name in os.listdir()]
    #print("total files in ", full_path,  " : " ,  len(folder_contents_names))
    
    # destination folder
    destination_path = "D:\\insta filter" 
    
    #making new folder in the destination folder for a hashtag
    destination_path = destination_path + "\\" + folder
    
    # making new hashtag folder for filtered data
    try:
        os.mkdir(destination_path)
    except OSError:
        print ("Creation of the directory %s failed" % destination_path)
    
    # creating  three lists for winrar, text file and image string names in the hashtag folder
    folder_onlywinrarstr=[]
    folder_onlytxtstr=[]
    folder_onlyjpgsstr=[]
    for name in folder_contents_names:
        if name.endswith('.xz'):
            folder_onlywinrarstr.append(name)
        if name.endswith('.txt'):
            folder_onlytxtstr.append(name)
        if name.endswith('.jpg'):
            folder_onlyjpgsstr.append(name)


    # counter to check progress
    counter = 1
    # total winrar files in the hashtag folder
    total_files_to_process= len(folder_onlywinrarstr)
    
    # iterarating all the winrar files in the hasg=htag foolder
    for single_winrar_file in folder_onlywinrarstr:

        print("Working on {} folder. File {} of {}".format(folder, counter ,total_files_to_process))
        counter+=1
        
        file_name=full_path+"\\"+single_winrar_file
        # opening a winrar json file, reading it in bytes and decoding it from bytes to string in utf 8
        # file_name something like D:\insta data\#fitnessgoals\2013-01-04_03-15-37_UTC.json.xz
        json_in_str= lzma.open(file_name).read().decode("utf-8")
        
        json_dict = json.loads(json_in_str)  # converting json from string into dictionary
        try:    
            address_data= json_dict['node']['location']['address_json'] # going to address key in dictionary
            address_data= address_data.replace('false','False') # changing false string in json to False datatype of python
            address_data= address_data.replace('true','True') # changing true string in json to True datatype of python 
            # address_data is string now. eval() will convert string to appropriate python datatype. Thats why false is made False and true is made True
            country_code = eval(address_data)['country_code']
            
            #print(country_code)
            #print(single_winrar_file)
            
            # if country code is empty make folder with name unknown
            if len(country_code) == 0:
                country_code = 'unknown'
            
            # taking just the winrar file name (without extension) if it has location data
            just_file_name = single_winrar_file[:-8]
            
            # path to folder that is to be created
            folder_to_create= destination_path + "\\"+ country_code
            
            #os.path.join("c:", os.sep, "Users", "Mainuser", "Desktop", "Lab6")
            
            # if country folder is not present, create a new folder for a new country
            if not os.path.isdir(folder_to_create):
                try:
                    os.mkdir(folder_to_create) # creating a new country folder
                    # there are multiple text files for a single name. few are old and few are new. saving both
                    for textfile in folder_onlytxtstr:
                        if just_file_name in textfile:
                            shutil.copy(textfile, folder_to_create)
                    # there are multiple images in a single instagram post. saving all the images of the post
                    for imagefile in folder_onlyjpgsstr:
                        if just_file_name in imagefile:
                            shutil.copy(imagefile, folder_to_create)
                except OSError:
                    print ("Creation of the directory %s failed" % folder_to_create)
            # if country folder is already present then save text file and image file to country specific folder
            else:
                # there are multiple text files for a single name. few are old and few are new. saving both
                for textfile in folder_onlytxtstr:
                    if just_file_name in textfile:
                        shutil.copy(textfile, folder_to_create)
                # there are multiple images in a single instagram post. saving all the images of the post
                for imagefile in folder_onlyjpgsstr:
                    if just_file_name in imagefile:
                        shutil.copy(imagefile, folder_to_create)        
        except:
            pass

In [ ]:
#Feature extraction VGG16 and KMeans clustering using Imagenet data

image.LOAD_TRUNCATED_IMAGES = True
model = VGG16(weights='imagenet', include_top=False)
# Main Dir after filter
maindir = 'D:/Insta filter/#healthgoals/'
folder =[]
os.chdir(maindir)
#Determining the country code in the folder
for name in os.listdir("."):
    if os.path.isdir(name):
        folder.append(name)

#Initialize no. of clusters to zero
number_clusters = 0
onlyfiles = []
#Determining the files in the country
for name in folder:
    ogdir = maindir + name +'/'
    os.chdir(ogdir)
    #changing the directory and finding jpg files
    for file in glob.glob("*.jpg"):
        onlyfiles.append(file)
    if len(onlyfiles) < 50:
        onlyfiles=[]
        continue #If the images are less than 50 we do not consider that folder
    elif (len(onlyfiles) > 50) and (len(onlyfiles) < 1000):
        number_clusters = 5 #For images between 50 and 1000 count we use no of clusters as 5
    else:
        number_clusters = 10 #For images more than 1000 no. of clusters is 10
    if number_clusters != 0:
        filelist = glob.glob(os.path.join(ogdir, '*.jpg'))
        filelist.sort() #files to extract features
        featurelist = []
        for i, path in enumerate(filelist):
            print("    Status: %s / %s" %(i, len(filelist)), end="\r")
            img = image.load_img(path, target_size=(224, 224)) #loading image 224x224 size
            data = image.img_to_array(img) 
            data = np.expand_dims(data, axis=0)
            data = preprocess_input(data)
            features = np.array(model.predict(data)) #Features extracted through transformation to array and dimension expansion
            featurelist.append(features.flatten()) #storing all features of the images
        kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(np.array(featurelist)) #Performing KMeans clustering on the features
        print("\n")
        targetdir = ogdir+"vgg16/" #Target Directory images being stored
        try:
            os.makedirs(targetdir)
        except OSError:
            pass
        for i, m in enumerate(kmeans.labels_):
            print("    Copy: %s / %s" %(i, len(kmeans.labels_)), end="\r")
            shutil.copy(filelist[i], targetdir + str(m) + "_" + str(onlyfiles[i])) #Copying the images to respective folders

        ac = AgglomerativeClustering(n_clusters=number_clusters, affinity='euclidean', linkage='ward').fit(np.array(featurelist))
        print("\n") #Performing Agglomerative Heirarichal Clustering using euclidean distance as the affinity
        targetdir = ogdir+"Agglo/"
        try:
            os.makedirs(targetdir)
        except OSError:
            pass
        for i, m in enumerate(ac.labels_):
            print("    Copy: %s / %s" %(i, len(ac.labels_)), end="\r")
            shutil.copy(filelist[i], targetdir + str(m) + "_" + str(onlyfiles[i])) #Copying images to respective folders
        onlyfiles=[] #Initializing no. of files to zero as swithing to new fodler

In [ ]:
#Extracting the top hashtags
#top_hash_tags.py

# setting main directoty where we have clustered countries
main_directory = 'D:/Insta filter/#healthgoals/'

os.chdir(main_directory)

#getting names of files in main folder
country_folders = [name for name in os.listdir()]

# dictionary to take all hashtags count
over_all_tag_count={}


hashtag= "#healthgoals"

print("Top 5 hashtags related to {}".format(hashtag))

# iterating each country
for country_name in country_folders:
    if os.path.isdir(country_name) and country_name != 'unknown':
        each_country_directory = main_directory + '//' + country_name
        # getting into each country folder
        os.chdir(each_country_directory)
        inside_country = os.getcwd()
        # selecting the clustered images folder
        vgg_folder_path = inside_country + '//' + 'vgg16' 
        if os.path.exists(vgg_folder_path):
            os.chdir(vgg_folder_path)
            print()
            # getting names of images
            clustered_images_names = [name for name in os.listdir()]
            
            # initial images cluster number is 0
            cluster_str_to_save = '0'
            # taking a variable for country
            country_str= country_name
            # dictionary to count hashtags counts for each country
            country_tag_count ={}
            
            os.chdir(inside_country)
            
            # iterating each image in each country
            for each_image in clustered_images_names:
                try:
                    # getting name of the image
                    just_image_name= each_image[2:25]
                    txt_file_name =  just_image_name + '.txt'
                    # checking for respective txt file with same image name
                    txt_content = open(txt_file_name,encoding="utf8") 
                    # reading the txt file
                    txt_content = txt_content.read()
                    # getting each word in the text file
                    words_in_txt = txt_content.split()
                    for word in words_in_txt:
                        # checking for # tags in the text files
                        if word.startswith("#"):
                            if word in country_tag_count:
                                country_tag_count[word] = country_tag_count[word]+1
                            else:
                                country_tag_count[word] = 1
                            if word in over_all_tag_count:
                                over_all_tag_count[word] = over_all_tag_count[word]+1
                            else:
                                over_all_tag_count[word] = 1
            
                    # getting the cluster number
                    cluster_str = each_image[:1]
                    if cluster_str_to_save != cluster_str:
                        country_tag_count_as_list = [country_tag_count]
                        
                        print(country_name)
                        print(cluster_str_to_save)
                        
                        # sort using a lambda expression to get top hash tags
                        sorted_names = sorted(country_tag_count, key=lambda x: country_tag_count[x])

                        # getting top 5 hash tags except the hashtag we are using to search for
                        val=0
                        for k in reversed(sorted_names):
                            if k != hashtag:
                                print("{} : {}".format(k, country_tag_count[k]))
    
                                val+=1
                                if val==5:
                                    break
                            
                        # getting the cluster number
                        cluster_str_to_save = cluster_str
                        # making dictionary empty to as to read for next country
                        country_tag_count={}
                        
                        
                        print()
                except:
                    continue

    os.chdir(main_directory)

# doing the dsame for all the text files irrespective of countries
print("For All Countries")
sorted_names = sorted(over_all_tag_count, key=lambda x: over_all_tag_count[x])


val=0
for k in reversed(sorted_names):
    if k != hashtag:
        print("{} : {}".format(k, over_all_tag_count[k]))
                
        val+=1
        if val==5:
            break

In [ ]:
#NEW FILE
# this is used to save the top hash tags in a text files by running the previous file and storing the output in a text file
import subprocess
with open("output.txt", "w+") as output:
    subprocess.call(["python", "top_hash_tags.py"], stdout=output)